<a href="https://colab.research.google.com/github/huavanson/CS114.K21/blob/master/Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset được lấy từ kaggle (https://www.kaggle.com/shree1992/housedata)

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sonhuavan","key":"899cfa81c0f3f4051aee96eb7e74b10f"}'}

In [2]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 65 Jul 15 03:45 kaggle.json


In [3]:
!kaggle datasets download -d shree1992/housedata

  0% 0.00/432k [00:00<?, ?B/s]
100% 432k/432k [00:00<00:00, 59.6MB/s]


In [4]:
!unzip housedata.zip

Archive:  housedata.zip
  inflating: data.csv                
  inflating: data.dat                
  inflating: output.csv              


import các thư viện cần thiết

In [5]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

Data có 4600 hàng và 18 cột 

In [6]:
data = pd.read_csv('data.csv')
print(data.shape)

(4600, 18)


In [8]:
data.head(5)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


Tiền xử lí : Chuyển các cột features thành dạng số 

In [9]:
from sklearn.preprocessing import LabelEncoder
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = data.columns.values.tolist()
for col in features:
    if data[col].dtype in numerics: continue
    categorical_columns.append(col)
# Encoding categorical features
for col in categorical_columns:
    if col in data.columns:
        le = LabelEncoder()
        le.fit(list(data[col].astype(str).values))
        data[col] = le.transform(list(data[col].astype(str).values))

Convert về cùng 1 kiểu int để dễ xử lí 

In [11]:
data['price'] = (data['price']).astype(int)
data['bathrooms']=(data['bathrooms']).astype(int)
data['floors'] = (data['floors']).astype(int)
data['bedrooms'] = (data['bedrooms']).astype(int)

Loại bỏ các cột bị mất dữ liệu (missing value )




In [46]:

data = data.drop(['waterfront'], axis=1)
data = data.drop(['sqft_basement'], axis=1)
data =  data.drop(['yr_renovated'],axis=1)
data = data.drop(['view'], axis=1)
data = data.drop(['country'], axis=1)
data.head(20)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,sqft_above,yr_built,street,city,statezip
0,313000,3,1,1340,7912,1,3,1340,1955,1522,36,62
1,2384000,5,2,3650,9050,2,5,3370,1921,3899,35,58
2,342000,3,2,1930,11947,1,4,1930,1966,2291,18,26
3,420000,3,2,2000,8030,1,4,1000,1963,4263,3,7
4,550000,4,2,1940,10500,1,4,1140,1976,4352,31,31
5,490000,2,1,880,6380,1,3,880,1938,3521,35,54
6,335000,2,2,1350,2560,1,3,1350,1976,2286,31,31
7,482000,4,2,2710,35868,2,3,2710,1989,2038,21,23
8,452500,3,2,2430,88426,1,4,1570,1985,3369,27,27
9,640000,4,2,1520,6200,1,3,1520,1945,3846,35,54


In [47]:
data.shape

(4600, 12)

Em chỉ lấy 1 số cột quan trọng làm features và label là price(giá nhà)

1 số features em xử dụng là số lượng phòng ngủ , phòng tắm trong căn nhà , diện tích, số tầng , năm xây dựng v.v . Em bỏ các cột địa chỉ, quốc gia ,v.v

Và chia theo tỷ lệ 80/20

In [60]:
from sklearn.model_selection import train_test_split
names=['bedrooms','bathrooms','sqft_living','sqft_lot','floors','condition','sqft_above','yr_built']
X=data[names]
y=data.price
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [61]:
X.shape

(4600, 8)

Gọi model LinearRegression để fit


In [62]:
model = LinearRegression() 
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Em sử dụng chỉ số MAE(mean_absolute_error) để đánh giá độ chính xác của model . 2 chỉ số MAE và MSE của thầy hướng dẫn đều giống nhau về mặt chức năng đó là đánh giá model nhưng em xin phép sử dụng MAE vì em học nó trên khóa học của Kaggle và sử dụng khá quen . chỉ số này hiểu đơn giản là đo độ lệch giữa giá trị predict và giá trị thực tế mà sử dụng trị tuyệt đối thay vì bình phương(MSE)

In [64]:
y_predict = model.predict(X_validation)
results = mean_absolute_error(y_validation,y_predict)
results

164387.34607879995

Chỉ số MAE khá tệ 